<a href="https://colab.research.google.com/github/jenieto/computer-vision/blob/integrate-google-colab/computer-vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a test!!

In [0]:
# Montamos Google Drive
from google.colab import drive
drive.mount('/content/drive/')
!unzip -o "/content/drive/My Drive/Datasets/computer-vision-M2.zip" -d /content/

In [0]:
# Importamos librerias
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import os 
from google.colab.patches import cv2_imshow

In [0]:
csv_path = '/content/anotaciones_itemsEvaluables_v3.csv'
images_path = '/content/dataset1'
figure = 'a1'

def read_data():
  data = pd.read_csv(csv_path, sep=';', names=['dir', 'image', 'figure', 'coords'], index_col=False, skipinitialspace=True)
  for key in data.keys():
    data[key] = data[key].apply(str.replace, args=(' ', ''))
  return data

def filter_figure(data, figure):
  filter = data['figure'] == figure
  return data[filter]

In [25]:
# Leemos los datos
data = read_data()
data = filter_figure(data, figure)
data

,dir,image,figure,coords
5099,REY_roi_manualSelection1,ROI_0_Psic_119Ev5b.pdf_pg-28.jpg,a1,"(29,445)"
5100,REY_roi_manualSelection1,ROI_0_Psic_205Ev4.pdf_pg-70.jpg,a1,"(34,24)"
5101,REY_roi_manualSelection1,ROI_0_Psic_101Ev5.pdf_pg-27.jpg,a1,"(27,41)"
5102,REY_roi_manualSelection1,ROI_90_Psic_029Ev4.pdf_pg-24.jpg,a1,"(37,33)"
5103,REY_roi_manualSelection1,ROI_0_Psic_516Ev3.pdf_pg-26.jpg,a1,"(39,41)"
...,...,...,...,...
5418,REY_roi_manualSelection1,ROI_0_Psic_517Ev3.pdf_pg-26.jpg,a1,"(35,50)"
5419,REY_roi_manualSelection1,ROI_0_Psic_408Ev2.pdf_pg-35.jpg,a1,"(29,34)"
5420,REY_roi_manualSelection1,ROI_0_Psic_210Ev5.pdf_pg-54.jpg,a1,"(33,39)"
5421,REY_roi_manualSelection1,ROI_0_Psic_514Ev3.pdf_pg-20.jpg,a1,"(34,33)"


In [0]:
def generate_images(data):
  images = []
  for index, row in data.iterrows():
    path = os.path.join(images_path, row['dir'], row['image'])
    image = cv2.imread(path)
    images.append(image)
  return np.array(images)

def generate_labels(data):
  labels = []
  for index, row in data.iterrows():
    coords_split = row['coords'].split('),(')
    coords_split = map(lambda x : x.replace('(','').replace(')',''), coords_split)
    row_coords = []
    for coords in coords_split:
      numbers = map(lambda x : int(x), coords.split(','))
      row_coords.extend(numbers)
    labels.append(row_coords)
  return np.array(labels)

In [0]:
# Generamos las imagenes y los outputs
images = generate_images(data)
labels = generate_labels(data)
labels